In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

In [7]:
import os

line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
open_api_key = os.environ.get('OPENAI_API_KEY')
port = 5000

In [8]:
from openai import OpenAI

client = OpenAI(api_key=open_api_key)
def get_completion(prompt):
    messages = [{"role": "system", "content": "以下用繁體中文台灣用語回答"}, 
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        max_tokens=256
        
    )
    return response.choices[0].message.content

In [3]:
print(get_completion("簡介東吳大學"))

東吳大學是一所位於台灣台北市的知名私立大學，創立於1928年，是台灣歷史悠久的學府之一。東吳大學以人文學科為主，並擁有豐富的師資和研究資源。校園環境優美，教學設備完善，是許多學生夢寐以求的就讀學校之一。該校提供多元的學士、碩士和博士學位課程，並致力於培養學生的專業知識和實務能力。東吳大學也積極推動國際交流與合作，為學生提供豐富的國際


In [9]:
messages = []

In [1]:
import os
from flask import Flask, request, abort
from openai import OpenAI
from linebot import (
    LineBotApi,
    WebhookHandler
)
from linebot.exceptions import InvalidSignatureError
from linebot.models import (
    MessageEvent,
    TextMessage,
    TextSendMessage,
    StickerMessage,
    StickerSendMessage,
    LocationMessage,
    LocationSendMessage,
    TemplateSendMessage,
    ButtonsTemplate,
    MessageAction
)

app = Flask(__name__)

# 從環境變數中獲取 LINE 和 OpenAI 的相關憑證
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
open_api_key = os.environ.get('OPENAI_API_KEY')
port = 5000

# 初始化 OpenAI 客戶端
openai_client = OpenAI(api_key=open_api_key)
taide_client = OpenAI(
    base_url='http://120.105.97.239:11434/v1/',
    api_key='ollama',
)

def get_gpt_completion(prompt):
    messages = [
        {"role": "system", "content": "以下用繁體中文台灣用語回答"},
        {"role": "user", "content": prompt}
    ]
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        max_tokens=1000
    )
    return response.choices[0].message.content

def get_taide_completion(prompt):
    messages = [
        {'role': 'system', 'content': '你是一個說繁體中文的助理，只會用台灣用語，不會用中國用語'},
        {'role': 'user', 'content': prompt}
    ]
    response = taide_client.chat.completions.create(
        model='jcai/llama3-taide-lx-8b-chat-alpha1:f16',
        messages=messages,
    )
    return response.choices[0].message.content

messages = []
is_ai_mode = False  # 標誌變量，跟踪是否處於 AI 對話模式
ai_choice = None  # 用戶選擇的 AI 模型

line_bot_api = LineBotApi(line_access_token)
handler = WebhookHandler(line_secret)

@app.route("/", methods=['POST'])
def callback():
    # 獲取 X-Line-Signature 標頭的值
    signature = request.headers['X-Line-Signature']

    # 獲取請求主體為文字
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # 處理 webhook 主體
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global is_ai_mode, ai_choice
    text = event.message.text
    if text == "3C助手":
        buttons_template = ButtonsTemplate(
            title='選擇 AI 模型',
            text='請選擇您想要使用的 AI 模型',
            actions=[
                MessageAction(label='GPT', text='選擇 GPT'),
                MessageAction(label='Taide', text='選擇 Taide')
            ]
        )
        template_message = TemplateSendMessage(
            alt_text='選擇 AI 模型',
            template=buttons_template
        )
        line_bot_api.reply_message(event.reply_token, template_message)
    elif text == "選擇 GPT":
        is_ai_mode = True
        ai_choice = "GPT"
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="已進入 GPT 對話模式，請輸入您的問題。輸入 '結束' 以退出。")
        )
    elif text == "選擇 Taide":
        is_ai_mode = True
        ai_choice = "Taide"
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="已進入 Taide 對話模式，請輸入您的問題。輸入 '結束' 以退出。")
        )
    elif text == "結束":
        is_ai_mode = False
        ai_choice = None
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="已退出 AI 對話模式。")
        )
    elif is_ai_mode:
        prompt = text
        messages.append(prompt)
        prompts = " ".join(messages)
        if ai_choice == "GPT":
            reply_text = get_gpt_completion(prompts)
        else:
            reply_text = get_taide_completion(prompts)
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=reply_text)
        )
        messages.append(reply_text)
    else:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=event.message.text)
        )

@handler.add(MessageEvent, message=StickerMessage)
def handle_sticker_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        StickerSendMessage(
            package_id=event.message.package_id,
            sticker_id=event.message.sticker_id
        )
    )

@handler.add(MessageEvent, message=LocationMessage)
def handle_location_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        LocationSendMessage(
            title='定位訊息',
            address=event.message.address,
            latitude=event.message.latitude,
            longitude=event.message.longitude
        )
    )

if __name__ == "__main__":
    app.run(port=port)


 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\USER\AppData\Local\Temp\ipykernel_25016\2508017174.py:65: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(line_access_token)
C:\Users\USER\AppData\Local\Temp\ipykernel_25016\2508017174.py:66: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(line_secret)
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


BODY:  {"destination":"U05f8fb23f03a1256e0ff3153571c9e24","events":[{"type":"message","message":{"type":"text","id":"511145343971754019","quoteToken":"7T28ExL5v_aUtPVMVUtN_u9KyVBJC9VOYswsI0WoKfMNZXHfp0jj48-u1Q9XRwWulV3jI1n0YNb1QzUdSw16OE7z1qg4IGEd7_w2Lbe9stp6piakUmJqH_XeOQ_SAuPJ6R9xydH7hAGb2FpAYl6Ybw","text":"3C助手"},"webhookEventId":"01HZHDDBGA8ZANSY387TFRAGS6","deliveryContext":{"isRedelivery":false},"timestamp":1717497670743,"source":{"type":"user","userId":"U41a3c59f7352195be95161a12b9f0917"},"replyToken":"56c28a0c9d1745dcb41ef8ace2f402ae","mode":"active"}]}


C:\Users\USER\AppData\Local\Temp\ipykernel_25016\2508017174.py:104: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, template_message)
127.0.0.1 - - [04/Jun/2024 18:41:11] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U05f8fb23f03a1256e0ff3153571c9e24","events":[{"type":"message","message":{"type":"text","id":"511145352494318134","quoteToken":"VDRlqzBHOf_f1Nt0tl1fmYiD_HT4T51dClYsVWk-AOqqALJj4gKeL-7dT2ixSkj41TJKf92OgRR9tJ3U7Snkxd2rM9o1_HlMMH-zY3aT2a0BNZ99K9s6DO1Ty6qabpKkXIK0uXwu4iDgmoehWogNTw","text":"選擇 GPT"},"webhookEventId":"01HZHDDG3S8XBA0TMB25Q3C6R1","deliveryContext":{"isRedelivery":false},"timestamp":1717497675656,"source":{"type":"user","userId":"U41a3c59f7352195be95161a12b9f0917"},"replyToken":"8fc19d05f2db409a81093f21daab6f1a","mode":"active"}]}


C:\Users\USER\AppData\Local\Temp\ipykernel_25016\2508017174.py:108: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(
127.0.0.1 - - [04/Jun/2024 18:41:16] "POST / HTTP/1.1" 200 -


: 